# Publicly-available Cebuano Corpus (An Exploration)

In [166]:
import json
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib
import numpy as np
import re
from tokenizers import normalizers, TextInputSequence, Tokenizer
from tokenizers.normalizers import NFD, NFKC, StripAccents, Lowercase, Sequence
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.trainers import BpeTrainer

## Wikipedia Corpus

In [41]:
text = []
for line in open('../text/AA/wiki_00', 'r'):
    text.append(json.loads(line))   

print first few lines

In [52]:
text[:2]

[{'id': '952',
  'url': 'https://ceb.wikipedia.org/wiki?curid=952',
  'title': 'Talaan sa mga lalawigan ug mga lungsod diin gitabi ang Binisayang Sinugboanon',
  'text': 'Talaan sa mga lalawigan ug mga lungsod diin gitabi ang Binisayang Sinugboanon\n\nKining maong panid talaan sa mga lalawigan ug mga lungsod diin gitabi ang Binisayang Sinugboanon.\n\nMga dakbayan: \n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga Lungsod\n\nMga lungsod\n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan:\n\nMga lungsod: \n\n\nMga dakbayan: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod:

In [127]:
re.sub(r'^(.*)\n\n*', '', text[0]['text'])

'Kining maong panid talaan sa mga lalawigan ug mga lungsod diin gitabi ang Binisayang Sinugboanon.\n\nMga dakbayan: \n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga Lungsod\n\nMga lungsod\n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan:\n\nMga lungsod: \n\n\nMga dakbayan: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\n\nMga lungsod: \n\n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga dakbayan: \n\nMga lungsod: \n\nMga lungsod: \n'

In [43]:
df = pd.DataFrame(text)
df['length'] = df['text'].apply(len)

In [44]:
print('Total current article count is', df.shape[0])
print('Total character length is', df.length.sum())

Total current article count is 43299
Total character length is 9561771


In [45]:
df['length'].describe()

count    43299.000000
mean       220.831220
std        642.739233
min        101.000000
25%        146.000000
50%        156.000000
75%        171.000000
max      60068.000000
Name: length, dtype: float64

### Training a BPE tokenizer for wiki corpus 

We first write the json file into a txt file line by line.

In [165]:
with open('../wiki-corpus.txt', 'w') as f:
    for line in text:
        _txt = re.sub(r'^(.*)\n\n*', '', line['text'])
        # remove redundant new lines
        _txt = re.sub(r'\n(?!$)', '', _txt)
        _txt = re.sub(r'\n\n', '\n', _txt)
        f.write(_txt)
    f.close()

In [167]:
tokenize_wiki = Tokenizer(BPE())
tokenize_wiki.normalizer = Sequence([
    NFD(),
    StripAccents(),
    Lowercase()
])
tokenize_wiki.pre_tokenizer = ByteLevel()
tokenize_wiki.decoder = ByteLevelDecoder()

In [168]:
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenize_wiki.train(["../wiki-corpus.txt"], trainer)

re-read the txt file

In [171]:
lines_wiki = []
with open('../wiki-corpus.txt', 'r') as f:
    for line in f:
        lines_wiki.append(line)


Let us check the tokenized inputs

In [174]:
for line in lines_wiki[:3]:
    encoded = tokenize.encode(line)
    print('token ids', encoded.ids)
    print('original line:', line)
    print('decoded:', tokenize.decode(encoded.ids))

token ids [1300, 752, 3620, 9944, 259, 285, 736, 288, 285, 549, 1191, 17548, 272, 11320, 11173, 13, 2490, 1006, 25, 285, 549, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 549, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 2490, 549, 2490, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 2490, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 1006, 25, 285, 549, 25, 285, 549, 25, 22793]
original line: Kining maong panid talaan sa mga lalawigan ug mga lungsod diin gitabi ang Binisayang Sinugboanon.Mga dakbayan: Mga lungsod: Mga lungsod: Mga dakbayan: Mga lungsod: Mga dakbayan: Mga lungsod: Mga lungsod: Mga lungsod: Mga dakbayan: Mga lungsod: Mga dakbayan: Mga LungsodMga lungsodMga dakbayan: Mga lungsod: Mga dakbayan: Mga lungsod: 

### Token Distribution

In [175]:
token_num = []
for line in lines_wiki:
    encoded = tokenize.encode(line)
    token_num.append(len(encoded.ids))

In [177]:
print('There are', np.sum(token_num), 'tokens in cebuano wiki data')

There are 2130156 tokens in cebuano wiki data


In [176]:
stats = pd.Series(token_num, name = 'stats').describe().to_frame()
stats

,stats
count,43148.000000
mean,49.368592
std,158.563136
min,11.000000
25%,31.000000
50%,34.000000
75%,38.000000
max,16336.000000


## OSCAR Cebuano corpus

In [5]:
lines_oscar = []
with open('../shuff-dedup/ceb/ceb_dedup.txt.gz', 'rb') as f:
    for line in f:
        lines_oscar.append(line.decode('utf-8'))


In [6]:
f'The OSCAR corpus contains about {len(lines_oscar)} lines of text.' 

'The OSCAR corpus contains about 75220 lines of text.'

In [7]:
lines_oscar[:2]

['matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado\n',
 'Maayong kaagahon liwat diri sa payag ICCC...nakaluy-an sa guihapon ang aton entry nga maka puntos sa ikaduha nga sulatada....goodluck sa madason pa guid nga mga sulatadas.\n']

### Create a pipeline

In [9]:
tokenize = Tokenizer(BPE())
tokenize.normalizer = Sequence([
    NFD(),
    StripAccents(),
    Lowercase()
])
tokenize.pre_tokenizer = ByteLevel()
tokenize.decoder = ByteLevelDecoder()

In [10]:
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenize.train(["../shuff-dedup/ceb/ceb_dedup.txt.gz"], trainer)

Take it for a spin, use it on the first three lines

In [11]:
for line in lines_oscar[:3]:
    encoded = tokenize.encode(line)
    print('token ids', encoded.ids)
    print('original line:', line)
    print('decoded:', tokenize.decode(encoded.ids))


token ids [11295, 319, 335, 70, 604, 555, 352, 419, 1313, 259, 631, 2391, 1772, 967, 11668, 291, 10979, 82, 11, 500, 8079, 2172, 410, 2299, 433, 291, 2936, 17611, 67, 335, 70, 604, 955, 3395, 545, 198]
original line: matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado

decoded:  matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado

token ids [1524, 2403, 22803, 6399, 3075, 259, 10312, 14266, 1130, 13544, 394, 88, 12, 256, 259, 16282, 1471, 272, 2743, 8416, 270, 458, 10354, 259, 3638, 270, 7168, 279, 306, 2391, 21522, 11592, 259, 282, 20652, 302, 10735, 270, 285, 7168, 279, 395, 13, 198]
original line: Maayong kaagahon liwat diri sa payag ICCC...nakaluy-an sa guihapon ang aton entry nga maka puntos sa ikaduha nga sulatada....goodluck sa madason pa guid nga mga sulatadas.

decoded:  maayong kaagahon liwat diri sa payag iccc...nakaluy-an sa guihapon ang aton entry nga maka 

In [12]:
token_num = []

for line in lines_oscar:
    encoded = tokenize.encode(line)  
    token_num.append(len(encoded.ids))


In [16]:
print('total number of tokens:', np.sum(token_num))

total number of tokens: 5496777


In [40]:
stats = pd.Series(token_num, name = 'stats').describe().to_frame()
stats



,stats
count,75220.000000
mean,73.076004
std,314.768952
min,2.000000
25%,38.000000
50%,63.000000
75%,84.000000
max,83901.000000
